## AMEX Default Competition - Feature Engineering


- This notebook does the following:

    - aggregating data by customer_ID
        - get the min, max, mean, std, first, last for each numeric features
            - then create a feature that checks if the last number if with 1.5 standard deviation of mean
        - get the unique count for categorical features 

    
- the raw data used for this notebook is generated by:
    - Process Amex Train Data to Parquet Format: https://www.kaggle.com/code/xxxxyyyy80008/process-amex-train-data-to-parquet-format
    - datasets can be accessed here:
        - train file: https://www.kaggle.com/datasets/xxxxyyyy80008/amex-train-20220706
        - test file: https://www.kaggle.com/datasets/xxxxyyyy80008/amex-test-20020706
    
- this notebook also used some insights from this notebook:
     - AMEX - Train Data EDA - Dask for Fast Analysis: https://www.kaggle.com/code/xxxxyyyy80008/amex-train-data-eda-dask-for-fast-analysis
     
- the data output of this notebook can be accessed here:
     - train data: https://www.kaggle.com/datasets/xxxxyyyy80008/amex-agg-data-rev2
   



In [1]:
import numpy as np
import pandas as pd
import gc
import copy
import os
import sys

from pathlib import Path
from datetime import datetime, date, time, timedelta
from dateutil import relativedelta

import pyarrow.parquet as pq
import pyarrow as pa

import dask.dataframe as dd

In [2]:
import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100


import pytorch_lightning as pl
random_seed=1234
pl.seed_everything(random_seed)

Global seed set to 1234


1234

## aggregate by customer id

In [3]:
all_cols = ['customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'D_49', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_63', 'D_64', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'D_66', 'B_20', 'D_68', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'D_73', 'P_4', 'D_74', 'D_75', 'D_76', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'R_9', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'B_29', 'B_30', 'S_18', 'D_86', 'D_87', 'R_17', 'R_18', 'D_88', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_89', 'R_22', 'R_23', 'D_91', 'D_92', 'D_93', 'D_94', 'R_24', 'R_25', 'D_96', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'B_36', 'B_37', 'R_26', 'R_27', 'B_38', 'D_108', 'D_109', 'D_110', 'D_111', 'B_39', 'D_112', 'B_40', 'S_27', 'D_113', 'D_114', 'D_115', 'D_116', 'D_117', 'D_118', 'D_119', 'D_120', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126', 'D_127', 'D_128', 'D_129', 'B_41', 'B_42', 'D_130', 'D_131', 'D_132', 'D_133', 'R_28', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145']

id_feats = ['customer_ID']
date_col =  'S_2'
num_cat_feats = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_66', 'D_68', 'B_31']
str_cat_feats = ['D_63', 'D_64', ]
float_feats = ['B_1', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_2', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_27', 'B_28', 'B_29', 'B_3', 'B_30', 'B_32', 'B_33', 'B_36', 'B_37', 'B_38', 'B_39', 'B_4', 'B_40', 'B_41', 'B_42', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'D_108', 'D_109', 'D_110', 'D_111', 'D_112', 'D_113', 'D_114', 'D_115', 'D_116', 'D_117', 'D_118', 'D_119', 'D_120', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126', 'D_127', 'D_128', 'D_129', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_65', 'D_66', 'D_68', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_84', 'D_86', 'D_87', 'D_88', 'D_89', 'D_91', 'D_92', 'D_93', 'D_94', 'D_96', 'P_2', 'P_3', 'P_4', 'R_1', 'R_10', 'R_11', 'R_12', 'R_13', 'R_14', 'R_15', 'R_16', 'R_17', 'R_18', 'R_19', 'R_2', 'R_20', 'R_21', 'R_22', 'R_23', 'R_24', 'R_25', 'R_26', 'R_27', 'R_28', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_17', 'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9']
int_feats = ['B_31']

print(len(float_feats))

185


In [4]:
len(all_cols)

190

In [4]:
%%time
# train_file = '/kaggle/input/amex-train-20220706/train.parquet'
# test_file = '/kaggle/input/amex-test-20020706/amex_test_20220706.parquet'

train_file = 'amex/train.parquet'
test_file = 'amex/amex_test_20220706.parquet'

Wall time: 0 ns


## Train file processing

In [5]:
eps =  1e-8

In [5]:
dest_folder = 'amex/train'

In [8]:
%%time
agg_files = []
stats = []
for c in float_feats:
    df = dd.read_parquet(train_file, columns=['customer_ID', 'S_2',c], engine='pyarrow')
    x = df.compute().sort_values(by='S_2', ascending=True).groupby("customer_ID").agg({c: ['min', 'max', 'mean', 'std', 'first','last']})
    x.columns = [f'{c1}__{c2}' for c1, c2 in x.columns]
    
    x[f'{c}__mean2std'] = (x[f'{c}__last'] >= (x[f'{c}__mean']-1.5*x[f'{c}__std'])) & (x[f'{c}__last'] <= (x[f'{c}__mean']+1.5*x[f'{c}__std']))
    x[f'{c}__mean2std'] = x[f'{c}__mean2std'].astype(int)
    
    x[f'{c}__last2max'] = (x[f'{c}__max']-x[f'{c}__last'])/(x[f'{c}__max'])
    x[f'{c}__last2min'] = (x[f'{c}__last']-x[f'{c}__min'])/(x[f'{c}__min'])
    x[f'{c}__range'] = (x[f'{c}__max']-x[f'{c}__min'])
    
    
    agg_cols = [f'{c}__min', f'{c}__max', f'{c}__mean', f'{c}__std', f'{c}__first', f'{c}__last', f'{c}__mean2std', 
                f'{c}__last2max', f'{c}__last2min', f'{c}__range']
    x.replace([-np.inf, np.inf], np.nan, inplace=True)
    
    for col_ in agg_cols:
        x[col_] = x[col_].fillna(value=x[col_].median())
        
    x[agg_cols] = np.float32(x[agg_cols].values)
    pq.write_table(pa.Table.from_pandas(x),  f'{dest_folder}/{c}.parquet', compression = 'GZIP')
    agg_files.append(f'{c}.parquet')
    
    #--calculate the stats ------------------------------------------------------------------

    stats.append([c] + x[agg_cols].median().values.tolist())

Wall time: 39min 22s


In [9]:
stats_cols = ['feat', 'min', 'max', 'mean', 'std', 'first','last', 
              'mean2std', 'last2max', 'last2min', 'range']

stats_df = pd.DataFrame(stats, columns= stats_cols )


stats_df.to_csv(f'{dest_folder}/train_agg_median.csv', index=False)

In [59]:
%%time
for c in str_cat_feats:
    df = dd.read_parquet(train_file, columns=['customer_ID', c], engine='pyarrow')
    df[c] = df[c].fillna(value='NA') 
    x0 = df.compute().groupby("customer_ID")[c].value_counts().to_frame()
    x0=x0.unstack().fillna(0)
    x0.columns = [f'{c0}={c1}' for c0, c1 in x0.columns]
    x0[x0.columns.tolist()] = np.float32(x0[x0.columns.tolist()].values)
    
    x1 = df.compute().groupby("customer_ID").agg({c: ['nunique']})
    x1.columns = [f'{c1}__{c2}' for c1, c2 in x1.columns]
    x1[x1.columns.tolist()] = np.int32(x1[x1.columns.tolist()].values)
    
    df = dd.read_parquet(train_file, columns=['customer_ID', 'S_2',c], engine='pyarrow')
    x2 = df.compute().sort_values(by='S_2', ascending=True).groupby("customer_ID").agg({c: ['last']})
    x2.columns = [f'{c1}__{c2}' for c1, c2 in x2.columns]
    col_ = x2.columns[0]
    x2[col_] = x2[col_].fillna(value='NA')
    dummies_ = pd.get_dummies(x2[col_])
    dummy_feats_ =  [f'{col_}={cc}' for cc in dummies_.columns]
    dummies_.columns = dummy_feats_
    x2[dummy_feats_] = dummies_.values
    x2.drop(columns=[col_], inplace=True)
    x2.fillna(value=0, inplace=True)
    
    x = x0.merge(x1, left_index=True, right_index=True, how='left')
    x = x.merge(x2, left_index=True, right_index=True, how='left')

    pq.write_table(pa.Table.from_pandas(x),  f'{dest_folder}/{c}.parquet', compression = 'GZIP')
    agg_files.append(f'{c}.parquet')

Wall time: 37.1 s


In [70]:
%%time
for c in int_feats + num_cat_feats:
    df = dd.read_parquet(train_file, columns=['customer_ID', c], engine='pyarrow')
    df[c] = df[c].fillna(value=999) 
    x0 = df.compute().groupby("customer_ID")[c].value_counts().to_frame()
    x0=x0.unstack().fillna(0)
    x0.columns = [f'{c0}={c1}' for c0, c1 in x0.columns]
    x0[x0.columns.tolist()] = np.float32(x0[x0.columns.tolist()].values)
    
    x1 = df.compute().groupby("customer_ID").agg({c: ['nunique']})
    x1.columns = [f'{c1}__{c2}' for c1, c2 in x1.columns]
    x1[x1.columns.tolist()] = np.int32(x1[x1.columns.tolist()].values)
    
    df = dd.read_parquet(train_file, columns=['customer_ID', 'S_2',c], engine='pyarrow')
    df[c] = df[c].fillna(value=999) 
    x2 = df.compute().sort_values(by='S_2', ascending=True).groupby("customer_ID").agg({c: ['last']})
    x2.columns = [f'{c1}__{c2}' for c1, c2 in x2.columns]
    
    x = x0.merge(x1, left_index=True, right_index=True, how='left')
    x = x.merge(x2, left_index=True, right_index=True, how='left')

    pq.write_table(pa.Table.from_pandas(x),  f'{dest_folder}/{c}.parquet', compression = 'GZIP')
    agg_files.append(f'{c}.parquet')

Wall time: 2min 47s


In [60]:
def cal_days(v):
    m0 = v['S_2=min']
    m1 = v['S_2=max']
    if m1 is np.nan:
        m1 = m0
    
    return (datetime.strptime(m1, '%Y-%m-%d') - datetime.strptime(m0, '%Y-%m-%d')).days

        

In [67]:
%%time
for c in ['S_2']:
    df = dd.read_parquet(train_file, columns=['customer_ID', c], engine='pyarrow')
    x = df.compute().groupby("customer_ID").agg({c: ['min', 'max', 'count']})
    x.columns = [f'{c1}={c2}' for c1, c2 in x.columns]   
    
    days = []
    for _, row in x.iterrows():
        days.append(cal_days(row))
    x['days']=days
    
    pq.write_table(pa.Table.from_pandas(x[['S_2=count', 'days']]), f'{dest_folder}/{c}.parquet', compression = 'GZIP')
    agg_files.append(f'{c}.parquet')

Wall time: 1min 18s


## combine all files

In [75]:
len(agg_files), agg_files[:1], len(set(agg_files))

(199, ['B_1.parquet'], 189)

In [76]:
agg_files = list(set(agg_files))

In [78]:
%%time
# df = pd.read_csv('/kaggle/input/amex-default-prediction/train_labels.csv')
df = pd.read_csv('amex/train_labels.csv')

# df = df[['customer_ID']].copy(deep=True)

for i, file in enumerate(agg_files):
    df_ = pd.read_parquet(f'{dest_folder}/{file}').reset_index()
    df = df.merge(df_,on=['customer_ID'], how='left')
        
    del df_
    gc.collect()
        
print(i)
pq.write_table(pa.Table.from_pandas(df), f'{dest_folder}/agg_train_all_rev3.parquet', compression = 'GZIP')
del df
gc.collect()

188


0

## log features

In [7]:
%%time
df = pd.read_parquet(f'{dest_folder}/agg_train_all_rev3.parquet', engine='pyarrow')


Wall time: 36.3 s


In [8]:
float_feats_ = list(set(float_feats)-set(num_cat_feats)-set(int_feats))
len(float_feats_)

176

In [9]:
all_cols2 = df.columns.tolist()
all_cols2.sort()
len(all_cols2)

1854

In [10]:
na_cnt = df.isna().sum()
na_cnt[na_cnt>0]

In [13]:
%%time
eps =  1e-8
log_feats = []

for c in all_cols2:
    
    if c in ['customer_ID', 'target']:
        continue
    
    if df[c].dtype not in ['float64', 'float32']:
        continue
        
    if '__' in c:
        c0, c1 = c.split('__')
        if (c0 in float_feats_) & (c1 in ['last', 'mean']):
            if df[c].min()>0:
                df[f'{c}__log'] = np.log(df[c].values + eps)
                df[f'{c}__log'].replace([-np.inf, np.inf], np.nan, inplace=True)
                df[f'{c}__log'] = df[f'{c}__log'].fillna(df[f'{c}__log'].median())
                log_feats.append(f'{c}__log')
            
            

Wall time: 2.34 s


In [14]:
float64_cols = df.select_dtypes(include=['float64']).columns.tolist()
df[float64_cols] = np.float32(df[float64_cols].values)
#---convert int64 to int32
int64_cols = df.select_dtypes(include=['int64']).columns.tolist()
df[int64_cols] = np.int32(df[int64_cols].values)

In [9]:
na_cnt = df.isna().sum()
na_cnt[na_cnt>0]

Series([], dtype: int64)

In [16]:
%%time
pq.write_table(pa.Table.from_pandas(df), f'{dest_folder}/agg_train_all_rev3.parquet', compression = 'GZIP')

Wall time: 3min 28s


## correlation with target

In [35]:
all_cols3 = df.columns.tolist()
all_cols3.sort()

all_cols3_ = list(set(all_cols3) - set(['customer_ID', 'S_2', 'target']))
all_cols3_.sort()
len(all_cols3), len(all_cols3_)

(2122, 2120)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 458913 entries, 0 to 458912
Columns: 2122 entries, customer_ID to S_9__mean__log
dtypes: float32(2094), int32(16), object(1), uint8(11)
memory usage: 3.6+ GB


In [37]:
%%time
corr_list = []

y = df['target'].values

for c in all_cols3_:
    x_ = df[c].values
    corr_list.append([c, np.corrcoef(x_, y)[0, 1]])

Wall time: 11.5 s


In [38]:
len(corr_list)

2120

In [ ]:
f'{dest_folder}/corr_w_target.csv'

In [39]:
corr_target = pd.DataFrame(data=corr_list, columns=['feat', 'corr'])
corr_target['corr_abs']=corr_target['corr'].abs()
corr_target.sort_values(by='corr_abs', ascending=False, inplace=True)
corr_target.to_csv(f'{dest_folder}/corr_w_target.csv', index=False)

In [6]:
corr_target = pd.read_csv('amex/train/corr_w_target.csv')

In [7]:
corr_target.head(660)

,feat,corr,corr_abs
0,P_2__last,-0.665749,0.665749
1,P_2__min,-0.634664,0.634664
2,P_2__mean,-0.625927,0.625927
3,D_48__last,0.602158,0.602158
4,P_2__max,-0.593064,0.593064
...,...,...,...
655,D_119__first,-0.169010,0.169010
656,D_122__mean,-0.169000,0.169000
657,R_12__std,0.168886,0.168886
658,R_13__last,0.167917,0.167917


In [8]:
topn_feats = corr_target.head(660)['feat'].values.tolist()
print(topn_feats)

['P_2__last', 'P_2__min', 'P_2__mean', 'D_48__last', 'P_2__max', 'R_1__mean__log', 'D_48__mean', 'R_1__std', 'D_44__mean__log', 'B_2__last__log', 'B_2__last', 'R_1__max', 'P_2__first', 'D_44__last__log', 'R_1__range', 'B_18__last', 'B_9__last', 'B_3__last__log', 'D_61__last', 'B_2__mean', 'D_48__max', 'D_55__last', 'B_9__mean', 'B_3__mean__log', 'R_1__mean', 'B_33__last', 'D_44__last', 'D_44__max', 'B_18__mean', 'D_48__min', 'B_23__last__log', 'B_33__mean', 'B_3__last', 'B_33__last__log', 'B_7__last', 'B_11__last__log', 'D_75__last', 'D_58__last', 'B_2__last2max', 'B_20__last__log', 'B_23__last', 'R_10__mean__log', 'B_23__mean__log', 'B_2__mean__log', 'D_55__max', 'D_61__mean', 'B_3__max', 'B_18__last__log', 'B_22__mean__log', 'D_75__mean__log', 'B_2__min', 'B_9__last__log', 'D_75__last__log', 'D_44__mean', 'B_11__mean__log', 'R_1__last', 'B_20__mean__log', 'D_75__max', 'R_10__std', 'B_18__min', 'B_16__last', 'D_61__min', 'D_78__mean__log', 'D_48__first', 'B_9__mean__log', 'D_58__max',

In [9]:
# corr_target[corr_target['corr_abs'].isna()]
corr_target[corr_target['corr_abs']>0.05]

,feat,corr,corr_abs
0,P_2__last,-0.665749,0.665749
1,P_2__min,-0.634664,0.634664
2,P_2__mean,-0.625927,0.625927
3,D_48__last,0.602158,0.602158
4,P_2__max,-0.593064,0.593064
...,...,...,...
1299,S_24__first,0.050525,0.050525
1300,D_125__min,0.050436,0.050436
1301,D_124__min,0.050322,0.050322
1302,D_60__first,-0.050313,0.050313


In [10]:
feats05 = corr_target[corr_target['corr_abs']>0.05]['feat'].values.tolist()
print(feats05)

['P_2__last', 'P_2__min', 'P_2__mean', 'D_48__last', 'P_2__max', 'R_1__mean__log', 'D_48__mean', 'R_1__std', 'D_44__mean__log', 'B_2__last__log', 'B_2__last', 'R_1__max', 'P_2__first', 'D_44__last__log', 'R_1__range', 'B_18__last', 'B_9__last', 'B_3__last__log', 'D_61__last', 'B_2__mean', 'D_48__max', 'D_55__last', 'B_9__mean', 'B_3__mean__log', 'R_1__mean', 'B_33__last', 'D_44__last', 'D_44__max', 'B_18__mean', 'D_48__min', 'B_23__last__log', 'B_33__mean', 'B_3__last', 'B_33__last__log', 'B_7__last', 'B_11__last__log', 'D_75__last', 'D_58__last', 'B_2__last2max', 'B_20__last__log', 'B_23__last', 'R_10__mean__log', 'B_23__mean__log', 'B_2__mean__log', 'D_55__max', 'D_61__mean', 'B_3__max', 'B_18__last__log', 'B_22__mean__log', 'D_75__mean__log', 'B_2__min', 'B_9__last__log', 'D_75__last__log', 'D_44__mean', 'B_11__mean__log', 'R_1__last', 'B_20__mean__log', 'D_75__max', 'R_10__std', 'B_18__min', 'B_16__last', 'D_61__min', 'D_78__mean__log', 'D_48__first', 'B_9__mean__log', 'D_58__max',